# **MicNet pipeline**
Micnet is a package developed for the analysis of microbial abundance table. It proposes an analysis pipeline which consists of three modules:
1. Visualization with UMAP and HDBSCAN
2. Estimation of the co-ocurrence network between species or OTUS with an enhanced version of SparCC.
3. Estimation of several graph theory metrics to describe the topology of the resulting network.

In this notebook we show an implementation of the analysis pipeline for the Komnbucha dataset described in (paper). 

For more information about the Micnet package: (paper)

If you prefer to use a GUI to implement most of the code presented here you can try our web app at: (link). And you can run this web app in your computer deploying the GUI locally with the code specified in our github repository (link).

## **Packages and dependancies**
### If you have not install the micnet package yet, please run the following code:

In [ ]:
pip install micnet==0.0.154

### Now we can import the packages

In [3]:
import hdbscan
import micnet as mc
import pandas as pd
from pathlib import Path
from bokeh.io import output_notebook
# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

Loading BokehJS ...

## **1.** **Load Kombucha data**

Now we will load and inspect the Kombucha data from the MicNet package. We can see that the data consist of 179 OTUs in the rows and in the columns we have the ASV id, the taxa classification and the abundance of 11 samples. 

In [ ]:
data = mc.load_kombucha()
print(data.shape)
data.head()

(179, 14)


,ASV_P,Taxa,ERR2139368,ERR2139369,ERR2139370,ERR2139371,ERR2139372,ERR2139373,ERR2139374,ERR2139375,ERR2139376,ERR2139377,ERR2139378,ERR2139379
0,ASV_P_1,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,3372.0,225.0,558.0,1405.0,256.0,3022.0,5302.0,3629.0,5569.0,3376.0,3690.0,4762.0
1,ASV_P_2,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,2373.0,705.0,530.0,3661.0,180.0,2229.0,3582.0,3670.0,4164.0,2559.0,2954.0,3090.0
2,ASV_P_3,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,2364.0,804.0,572.0,596.0,621.0,2741.0,4037.0,3156.0,4136.0,2842.0,3418.0,4230.0
3,ASV_P_4,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,2231.0,657.0,1954.0,4031.0,762.0,1799.0,743.0,848.0,3475.0,2142.0,2181.0,2290.0
4,ASV_P_5,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,1997.0,919.0,2215.0,2580.0,145.0,1348.0,2551.0,2482.0,3393.0,626.0,281.0,483.0


## **1.1 Pre-processing the data**
We have the option of filtering out singletons and low-abundace OTUs.In this case we will do it to have a more reliable dataset to work with.

In [ ]:
# the filtering functions needs you to specify if the input data contains any taxa information
from micnet.utils import filter_otus
X,Taxa,Text = filter_otus(data,taxa=True,low_abundance=True)
print(X.shape)
X

(48, 12)


,ERR2139368,ERR2139369,ERR2139370,ERR2139371,ERR2139372,ERR2139373,ERR2139374,ERR2139375,ERR2139376,ERR2139377,ERR2139378,ERR2139379
0,3372.0,225.0,558.0,1405.0,256.0,3022.0,5302.0,3629.0,5569.0,3376.0,3690.0,4762.0
1,2373.0,705.0,530.0,3661.0,180.0,2229.0,3582.0,3670.0,4164.0,2559.0,2954.0,3090.0
2,2364.0,804.0,572.0,596.0,621.0,2741.0,4037.0,3156.0,4136.0,2842.0,3418.0,4230.0
3,2231.0,657.0,1954.0,4031.0,762.0,1799.0,743.0,848.0,3475.0,2142.0,2181.0,2290.0
4,1997.0,919.0,2215.0,2580.0,145.0,1348.0,2551.0,2482.0,3393.0,626.0,281.0,483.0
5,2162.0,662.0,1672.0,2657.0,2793.0,1020.0,202.0,2541.0,327.0,988.0,2395.0,113.0
6,3071.0,1981.0,2482.0,383.0,3595.0,1450.0,286.0,569.0,266.0,384.0,1781.0,568.0
7,2183.0,548.0,446.0,3100.0,1590.0,289.0,905.0,2600.0,1178.0,945.0,1971.0,282.0
8,709.0,859.0,2782.0,456.0,680.0,941.0,1142.0,733.0,1466.0,867.0,1183.0,1482.0
9,1477.0,196.0,1426.0,540.0,312.0,1026.0,2237.0,437.0,2284.0,1105.0,340.0,1866.0


## **2. Visualize the data with UMAP and HDBSCAN**

To visualize our abundance table using dimension reduction technique UMAP and HDBSCAN for clustering we need to create a class called **Embedding_Ouput**. This is where we have to decide on the parameters of umap and hdbscan. For the Kombucha example we will set them as in the paper by Favila et al (2021).

In [ ]:
#Set parameter values
#The metrisc for umap and hdbscan can be picked from the following:
METRIC_UMAP=['euclidean','manhattan','canberra','braycurtis', 'cosine','correlation','hellinger']
METRIC_HDB=['euclidean','manhattan','canberra','braycurtis']
n_neighbors = 2
min_dist = 1
n_components = 2 
metric_umap = METRIC_UMAP[0]
metric_hdb = METRIC_HDB[0]
min_cluster_size = 2
min_sample = 3

embedding_outliers=mc.Embedding_Output(n_neighbors=n_neighbors,min_dist=min_dist, n_components=n_components,
                                    metric_umap=metric_umap,metric_hdb=metric_hdb,min_cluster_size=min_cluster_size,
                                    min_sample=min_sample,output=True)

Now we the object created we can obtained the two dimensions from the UMAP reduction analysis and then we proceed to plot it in a ciruclar arrangement.

In [ ]:
embedding,o,l=embedding_outliers.fit(X)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_spectral_embedding.py:261: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  "Graph is not fully connected, spectral embedding may not work as expected."


In [ ]:
mc.plot_umap(embedding,l,Text,Taxa)

To have all the data of which OTU belongs to which cluster and if it is an outlier or not we can put all teh data together in a dataframe:

In [ ]:
DF=pd.DataFrame()
if len(Taxa)>1:
    DF['Taxa']= Text.iloc[:,1]
DF['Outliers']=o
DF['Cluster']=l
DF.head()

,Taxa,Outliers,Cluster
0,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,0,2
1,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,0,2
2,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,0,2
3,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,0,-1
4,Bacteria;Proteobacteria;Alphaproteobacteria;Ac...,0,0


## **3. SparCC: calculating co-occurrence network**

To run SparCC on our abundace table we first need to instatiate the class SparCC_MicNet with the values of the parameters that we wish Sparcc is run with as follows:


In [ ]:
#SparCC is run wihtout any ASV or taxa so we first clean our dataset
dataSparcc = data.iloc[:,2:]
print(dataSparcc.shape)
dataSparcc.head()

(179, 12)


,ERR2139368,ERR2139369,ERR2139370,ERR2139371,ERR2139372,ERR2139373,ERR2139374,ERR2139375,ERR2139376,ERR2139377,ERR2139378,ERR2139379
0,3372.0,225.0,558.0,1405.0,256.0,3022.0,5302.0,3629.0,5569.0,3376.0,3690.0,4762.0
1,2373.0,705.0,530.0,3661.0,180.0,2229.0,3582.0,3670.0,4164.0,2559.0,2954.0,3090.0
2,2364.0,804.0,572.0,596.0,621.0,2741.0,4037.0,3156.0,4136.0,2842.0,3418.0,4230.0
3,2231.0,657.0,1954.0,4031.0,762.0,1799.0,743.0,848.0,3475.0,2142.0,2181.0,2290.0
4,1997.0,919.0,2215.0,2580.0,145.0,1348.0,2551.0,2482.0,3393.0,626.0,281.0,483.0


In [ ]:
# set parameters for SparCC
n_iteractions=3
x_iteractions=3
low_abundance=True
threshold=0.1
normalization='dirichlet'
log_transform=True
num_simulate_data=5
type_pvalues='one_sided'

#Create Sparcc object
SparCC_MN = mc.SparCC_MicNet(n_iteractions=n_iteractions,
                                    x_iteractions=x_iteractions,
                                    low_abundance=low_abundance,
                                    threshold=threshold,
                                    normalization=normalization,
                                    log_transform=log_transform,
                                    num_simulate_data=num_simulate_data,
                                    type_pvalues=type_pvalues,
                                    )

Then we actually run the sparcc algorithm with the method run_all

In [ ]:
SparCC_MN.run_all(data_input=dataSparcc)

sparcc will compute the correlations and the pvalues separately:

In [ ]:
DF_SparCC=pd.read_csv(Path(SparCC_MN.save_corr_file).resolve(),index_col=0)
DF_PValues=pd.read_csv(Path(SparCC_MN.outfile_pvals).resolve(),index_col=0)

So we can obtain the final significant correlations found by filtering them out by their p-value:

In [ ]:
sparcc_corr=DF_SparCC[DF_PValues<0.05].fillna(0)
print(f'The resulting corellation matrix is of size {sparcc_corr.shape}')
sparcc_corr.head() 

The resulting corellation matrix is of size (48, 48)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.000000,0.944201,0.923202,0.665196,0.467125,-0.168606,0.000000,0.000000,0.529983,0.768857,0.497204,-0.334890,0.660191,0.908319,0.250341,0.000000,-0.441019,0.000000,0.603738,0.000000,0.00000,0.000000,0.596691,-0.534202,0.000000,0.000000,-0.513746,0.467654,0.000000,-0.483057,-0.130440,0.000000,0.0,-0.450173,-0.142738,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.309089,0.000000,-0.414576,0.641005,0.000000,0.043613,0.297838
1,0.944201,0.000000,0.814922,0.640846,0.500614,0.000000,-0.263074,0.000000,0.387429,0.616227,0.477764,-0.358078,0.569799,0.861133,0.000000,0.497564,-0.516398,0.000000,0.500151,0.198715,0.46348,0.000000,0.751127,-0.633662,0.000000,0.000000,-0.655678,0.322413,0.000000,-0.489935,0.000000,-0.398891,0.0,-0.330998,-0.286595,-0.198283,0.000000,-0.099182,0.0,0.000000,0.220740,0.000000,0.000000,-0.459324,0.543778,0.000000,0.000000,0.000000
2,0.923202,0.814922,0.000000,0.570386,0.231810,-0.213453,0.000000,0.151919,0.654432,0.725490,0.000000,0.000000,0.771152,0.797236,0.000000,0.000000,-0.292974,0.000000,0.565728,0.000000,0.00000,0.000000,0.000000,-0.560854,-0.421608,0.278219,-0.560456,0.400982,-0.375806,-0.594219,-0.076714,-0.480959,0.0,-0.491106,0.000000,0.000000,-0.423685,0.000000,0.0,0.000000,0.000000,-0.226517,0.321667,-0.497361,0.762307,0.000000,0.000000,0.000000
3,0.665196,0.640846,0.570386,0.000000,0.296895,0.000000,0.000000,0.273016,0.680054,0.706513,0.710438,0.000000,0.000000,0.727497,0.547022,0.761272,0.086073,-0.483434,0.828881,0.346141,0.00000,0.430813,0.000000,0.000000,-0.647133,0.000000,-0.281950,0.681418,-0.623844,-0.428971,0.000000,-0.051356,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-0.498875,0.000000,-0.428443,0.338713,-0.443517,0.783405,0.259862,0.000000,0.439134
4,0.467125,0.500614,0.231810,0.296895,0.000000,0.000000,-0.388430,-0.346396,0.000000,0.545677,0.429794,0.000000,0.483728,0.565698,0.000000,0.379663,-0.389166,0.000000,0.000000,0.000000,0.00000,0.000000,0.670852,0.000000,0.000000,0.000000,-0.264274,0.000000,-0.425466,-0.148567,0.000000,-0.239411,0.0,0.000000,0.000000,0.000000,0.000000,-0.271626,0.0,0.154694,0.136324,0.043292,-0.492806,0.000000,0.000000,0.269689,0.000000,0.733227


## **4.Network analysis**

The final step on the proposed analysis pipeline of MicNet is to obtain large scale metrics of the network and subgroups based on their relationships. 

To do this we begin by building the graph of the matrix obtained from SparCC to be able to get large-scale metrics. Note that we can do this in two ways, by normalizing the correlation values to a range of (0,1) or we can leave the values as they are, this is your decision, but note that some graph theory analysis only work with normalized values.

In [ ]:
M = mc.build_network(sparcc_corr)
Mnorm = mc.build_normalize_network(sparcc_corr)

Now we can create a network micnet obejct which we will use to obtain most of our descriptors and analysis of the network

In [ ]:
NetM=mc.NetWork_MicNet()

Les start by looking at some of the basic properties of the network, such as number of nodes, number of interactions, diameter, etc..

In [ ]:
NetM.basic_description(corr=sparcc_corr)

Now we might be interested in how many triads including different types (such as  + + + or - + -) of interactions are present ir our network. We can do this by calling the structural balance method:

In [ ]:
NetM.structural_balance(M)

Remember that structural balance anaylsis needs the raw data (ranging from -1 to 1), not the normalized ones, that is why in this analysis we used the network M.

Now we can obtain the communties found in the network based on the Louvain method (which finds clusters based on increasing intragroup interactions and minimizing intergroup interactions)

In [ ]:
Communities=NetM.community_analysis(Mnorm)

Within the communities object we can extract the number of comminities and a table with a summary of properties of each community found as we show below:

In [ ]:
print(Communities['Number'])
print(Communities['Community_topology'])

Finally, it is also possible to extract the assignement of each node to the different communities found:

In [ ]:
print(Communities['Data'].head())

Now we obtain the centrlities of the nodes:

In [ ]:
Centrality=NetM.key_otus(Mnorm)

For easier display we can put everything in a single database, including the info that we previously obtained from  hdbscan clustering:

In [ ]:
NetDF=pd.DataFrame({'OTUS':Centrality['NUM_OTUS'],
              'Degree_Centrality':Centrality['Degree centrality'],
              'Betweeness_Centrality':Centrality['Betweeness centrality'],
              'Closeness_Centrality':Centrality['Closeness centrality'],
              'PageRank':Centrality['PageRank'],
              'HDBSCAN':DF['Cluster'],
              'Community':Communities['Community_data'].values.ravel()})

We now plot the network, coloring by communities:

In [ ]:
mc.plot_bokeh(graph=M,frame=SparrDF,
              nodes = M.number_of_nodes(),
              max = DF_Output.max().max(),
              min = DF_Output.min().min(),
              kind_network='spring',
              kind='Community')

We could also colored the nodes according to the groups found with HDBSCAN and in a circular layout:

In [ ]:
mc.plot_bokeh(graph=M,frame=SparrDF,
              nodes = M.number_of_nodes(),
              max = DF_Output.max().max(),
              min = DF_Output.min().min(),
              kind_network='circular',
              kind='HDBSCAN')

### 4.1 Topology comparison

### 4.2 Percolation analysis